In [25]:
import numpy as np
import pandas as pd
import boto3
from decimal import Decimal
import datetime as dt

# Insert Movies Table

In [27]:
movies = pd.read_csv('./movies.csv', dtype={0:'int64',1:'object', 2:'object'})
links = pd.read_csv('./links.csv', dtype={0:'int64',1: 'object', 2:'object'})
links.set_index('movieId', inplace=True)

In [31]:
movies.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [29]:
with client.Table('movies').batch_writer() as batch:
    for index, movie in movies.iterrows():
        data = {
            "movieId": movie["movieId"],
            "title": movie["title"],
            "imdbId": links.loc[movie["movieId"],"imdbId"],
            "url": "https://www.imdb.com/title/tt" + str(links.loc[movie["movieId"],"imdbId"]),
            "genres": set(movie["genres"].split("|"))
        }
        batch.put_item(Item=data)
    

# Insert Ratings Table

In [23]:
ratings = pd.read_csv('./ratings.csv', dtype={0:'int64',1:'int64',2:'object',3:'int64'})

In [13]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [24]:
with client.Table('movie_ratings').batch_writer() as batch:
    for index, rating in ratings.iterrows():
        data = {
            "userId": rating["userId"],
            "movieId": rating["movieId"],
            "rating": Decimal(rating["rating"]),
            "timestamp": rating['timestamp']
        }
        batch.put_item(Item=data)

KeyboardInterrupt: 